In [3]:
from i_o import Paths
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os, glob, h5py
import importlib, pathlib
from daio.h5 import lazyh5
from video import create_projection_image, showvid
import warpfield
from IPython.display import JSON

/home/lubo12/.conda/envs/lfm1/lib/python3.12/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


In [4]:
pn_out = r"~/hpc-rw/lfm/results"
includes = ['']
excludes = []

recs = [os.path.basename(rec) for rec in sorted(glob.glob(os.path.expanduser(os.path.join(pn_out, '*'))), reverse=True) if os.path.exists(os.path.join(pn_out, rec, 'deconvolved.h5'))]
recs = [rec for rec in recs if all([s in rec for s in includes]) and not any([s in rec for s in excludes])]
print('\n'.join(recs))

20250701_1625_LB_ZF_v552_f7_40fps_2
20250701_1622_LB_ZF_v552_f7_40fps_1
20250701_1605_LB_ZF_v552_f6_40fps_1
20250701_1538_LB_ZF_v552_f5_40fps_1
20250514_1635_LF_ZF552_f4_1_comp_40fps
20250514_1515_LF_ZF552_f1_4_uncomp_40fps
20250513_1351_LB_redFB_1-1k_3d_with_movement


In [5]:
pn_psfs = r"~/hpc-rw/lfm/psfs"
pn_rec = r"~/hpc-r/lfm2025/recordings"
pn_bg = r"~/hpc-rw/lfm/bg"
url_home = r"/home/lubo12/"
pn_out = r"~/hpc-rw/lfm/results"

dataset_name = "20250701_1605_LB_ZF_v552_f6_40fps_1"
psf_name = "20250701_1216_PSF_LB_noap_1"
bg_name = "20250701_1651_LB_bg_40fps.npy"

paths = Paths(dataset_name=dataset_name,
              psf_name = psf_name,
              bg_name=bg_name,
              pn_bg= pn_bg,
              pn_rec = pn_rec,
              pn_psfs=pn_psfs,
              pn_out=pn_out,
              url_home=url_home,
              )

In [6]:
#generate default registration recipe
recipe = warpfield.Recipe.from_yaml('default.yml')

### Generate reference volume from recording

In [13]:
import register, i_o
importlib.reload(register)
from register import generate_reference_volume, refine_reference_volume
importlib.reload(i_o)
from i_o import volume_reader

In [16]:
reader = volume_reader(paths.deconvolved, "data", np.arange(0,10,1))

AttributeError: 'Dataset' object has no attribute 'append'

In [23]:
lazyh5("test.h5")

<IPython.core.display.JSON object>

In [ ]:
#inspect theindexes to use for generating the refenrence - should be very little movement
ref_idx = [0,100,1]
ref_vol_unreg, video_fn = reference_volume(paths,
                                              ref_idx,
                                              vmax=100,
                                              vmin=0,
                                              absolute_limits=False)

showvid(video_fn)

In [ ]:
plt.imshow(create_projection_image(ref_vol_unreg, vmax=100, vmin=0, absolute_limits=False)

In [ ]:
#refine reference volume by registering every volume to the average and then averaging again
def register_reference(vol, ref_vol, register_recipe):
    return warpfield.register(vol, ref_vol, register_recipe)


ref_vol, video_fn = reference_volume(paths,
                                      ref_idx,
                                     preprocess = register_reference
                                      vmax=100,
                                      vmin=0,
                                      absolute_limits=False,
                                    register_recipe=register_recipe,
                                    ref_vol=
                                      )

showvid(video_fn)

In [ ]:
#adjust registration params

### Test mini registration